In [1]:
with open('input.txt', 'r') as input_file:
    text = input_file.read()

In [2]:
unique_chars = sorted(list(set(text)))
vocab_size = len(unique_chars)
print(''.join(unique_chars))
print(len(unique_chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz
65


In [3]:
ctoi = { char:ind for ind,char in enumerate(unique_chars)}
itoc = {ind:char for ind,char in enumerate(unique_chars)}

encode = lambda s: [ctoi[z] for z in s]
decode = lambda i: ''.join(itoc[z] for z in i)

print(encode('hello'))
print(decode(encode('hello')))

[46, 43, 50, 50, 53]
hello


In [4]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = torch.tensor(encode(text), dtype = torch.long)
data[:100]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])

In [6]:
# train and test split. we are considering 90% of the data as our training data.

factor = int(0.9 * len(data))
train = data[:factor]
val = data[factor:]

In [7]:
blk = 8
print(train[:blk],'\n')

def get_targets(mat, tar, block_size):
    for _ in range(block_size):
        context = mat[:_+1]
        target = tar[_]
        print(f'context:{context}',f'target: {target}\n')


tensor([18, 47, 56, 57, 58,  1, 15, 47]) 



In [8]:
torch.manual_seed(1337)
batch_size = 4
blk_size = 8

def get_batch(split):
    data = train if split == 'train' else val
    indices = torch.randint((len(data) - blk_size), (batch_size,))
    x = torch.stack([data[i:i+blk_size] for i in indices])
    y = torch.stack([data[i+1:i+blk_size+1] for i in indices])
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')

x.shape,y.shape,x,y

(torch.Size([4, 8]),
 torch.Size([4, 8]),
 tensor([[24, 43, 58,  5, 57,  1, 46, 43],
         [44, 53, 56,  1, 58, 46, 39, 58],
         [52, 58,  1, 58, 46, 39, 58,  1],
         [25, 17, 27, 10,  0, 21,  1, 54]]),
 tensor([[43, 58,  5, 57,  1, 46, 43, 39],
         [53, 56,  1, 58, 46, 39, 58,  1],
         [58,  1, 58, 46, 39, 58,  1, 46],
         [17, 27, 10,  0, 21,  1, 54, 39]]))

In [9]:
for _ in range(batch_size):
    get_targets(x[_],y[_],len(x[_]))
    

context:tensor([24]) target: 43

context:tensor([24, 43]) target: 58

context:tensor([24, 43, 58]) target: 5

context:tensor([24, 43, 58,  5]) target: 57

context:tensor([24, 43, 58,  5, 57]) target: 1

context:tensor([24, 43, 58,  5, 57,  1]) target: 46

context:tensor([24, 43, 58,  5, 57,  1, 46]) target: 43

context:tensor([24, 43, 58,  5, 57,  1, 46, 43]) target: 39

context:tensor([44]) target: 53

context:tensor([44, 53]) target: 56

context:tensor([44, 53, 56]) target: 1

context:tensor([44, 53, 56,  1]) target: 58

context:tensor([44, 53, 56,  1, 58]) target: 46

context:tensor([44, 53, 56,  1, 58, 46]) target: 39

context:tensor([44, 53, 56,  1, 58, 46, 39]) target: 58

context:tensor([44, 53, 56,  1, 58, 46, 39, 58]) target: 1

context:tensor([52]) target: 58

context:tensor([52, 58]) target: 1

context:tensor([52, 58,  1]) target: 58

context:tensor([52, 58,  1, 58]) target: 46

context:tensor([52, 58,  1, 58, 46]) target: 39

context:tensor([52, 58,  1, 58, 46, 39]) target:

In [23]:
import torch.nn as nn
from torch.nn import functional as f
n_embd = 32

In [64]:
#single head of self attention

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias = False)
        self. query = nn.Linear(n_embd,head_size,bias = False)
        self.value = nn.Linear(n_embd,head_size,bias = False)
        self.register_buffer('tril',torch.tril(torch.ones(blk_size,blk_size)))

    def forward(self, x):
        a,b,c = x.shape
        k = self.key(x)
        q = self.query(x)
        v = self.value(x)
        
        z = q @ k.transpose(-2, -1) * c** -0.5
        z = z.masked_fill(self.tril[:b,:b] == 0, float('-inf'))
        z = f.softmax(z,dim = -1)
        out = z@v

        return out
        

In [80]:
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.projection = nn.Linear(n_embd,n_embd)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads],dim = -1)
        out = self.projection(out)
        
        return out
        

In [81]:
class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd,n_embd)
        )

    def forward(self, x):
        return self.net(x)

In [82]:
class Block(nn.Module):
    def __init__(self, n_embd, n_heads):
        super().__init__()
        head_size = n_embd//n_heads
        self.sa = MultiHeadAttention(n_heads, head_size)
        self.ff = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd,n_embd)       
        self.ln2 = nn.LayerNorm(n_embd,n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ff(self.ln2(x))
        return x
        

In [86]:
torch.manual_seed(1337)

class BigramModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.embedding_table = nn.Embedding(vocab_size, n_embd)
        self.pos_embedding_table = nn.Embedding(blk_size, n_embd)
        # self.sa_head = Head(n_embd)
        
        # self.sa_heads = MultiHeadAttention(4, n_embd//4)     
        #self.ff = FeedForward(n_embd)

        self.blocks = nn.Sequential(
            Block(n_embd, n_heads = 4),
            Block(n_embd, n_heads = 4),
            Block(n_embd, n_heads = 4),
            nn.LayerNorm(n_embd,vocab_size)         
        )
        
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        B,T = idx.shape
        token_embd = self.embedding_table(idx)
        pos_embd = self.pos_embedding_table(torch.arange(T,device = device))
        x = token_embd+pos_embd
        #x = self.sa_heads(x)
        #x = self.ff(x)
        x = self.blocks(x)
        
        logits = self.lm_head(x)
        
        if targets is None:
            loss = None
        else :
            B,T,C = logits.shape
            logits = logits.view(B*T,C)
            targets = targets.view(B*T)
            loss = f.cross_entropy(logits,targets)
            
        return loss,logits

    def generate(self, idx, max_tokens):
        for _ in range(max_tokens):
            idx_ = idx[:,-blk_size:]
            loss,logits = self(idx)
            logits = logits[:,-1,:]  
            probs = f.softmax(logits, dim = 1)          
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next), dim = 1)
        return idx

model = BigramModel()
model = model.to(device)

idx = torch.zeros((1,1), dtype = torch.long, device = device)
print(model.generate(idx,max_tokens=8))
print(decode(model.generate(idx,max_tokens=8)[0].tolist()))
        

tensor([[ 0, 12,  0, 44, 55, 10, 64, 48, 12]])

yVnq;O:$


In [87]:
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3)

In [88]:
batch_size = 32;

for _ in range(5000):
    x,y = get_batch('train')
    loss,logits = model(x,y)
    
    optimizer.zero_grad(set_to_none = True)
    loss.backward()
    optimizer.step()

print(loss.item())

1.9110002517700195


In [30]:
torch.manual_seed(1337)
a,b,c = 4,8,32
x = torch.randn(a,b,c)
x

tensor([[[ 0.1808, -0.0700, -0.3596,  ..., -0.8016,  1.5236,  2.5086],
         [-0.6631, -0.2513,  1.0101,  ...,  1.5333,  1.6097, -0.4032],
         [-0.8345,  0.5978, -0.0514,  ..., -0.4370, -1.0012, -0.4094],
         ...,
         [-0.8961,  0.0662, -0.0563,  ...,  2.1382,  0.5114,  1.2191],
         [ 0.1910, -0.3425,  1.7955,  ...,  0.3699, -0.5556, -0.3983],
         [-0.5819, -0.2208,  0.0135,  ..., -1.9079, -0.5276,  1.0807]],

        [[ 0.4562, -1.0917, -0.8207,  ...,  0.0512, -0.6576, -2.5729],
         [ 0.0210,  1.0060, -1.2492,  ...,  0.7859, -1.1501,  1.3132],
         [ 2.2007, -0.2195,  0.5427,  ..., -0.6445,  1.0834, -0.7995],
         ...,
         [ 0.3091,  1.1661, -2.1821,  ...,  0.6151,  0.6763,  0.6228],
         [ 0.0943, -0.3156,  0.7850,  ..., -1.5735,  1.3876,  0.7251],
         [ 0.6455, -0.3313, -1.0390,  ...,  0.0895, -0.3748, -0.4781]],

        [[-0.6067,  1.8328,  0.2931,  ...,  1.0041,  0.8656,  0.1688],
         [-0.2352, -0.2586,  0.0131,  ...,  0

In [16]:
#brute force
res1 = torch.zeros(a,b,c)
for row in range(a):
    for col in range(b):
        prev = x[row,:col+1]
        mean = torch.mean(prev, dim = 0)
        res1[row,col] = mean
res1[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [17]:
# vectorized
tril = torch.tril(torch.ones(b,b))
t = tril/torch.sum(tril,dim = 1,keepdims = True)
res2  = t@x
res2[0]

tensor([[ 0.1808, -0.0700],
        [-0.0894, -0.4926],
        [ 0.1490, -0.3199],
        [ 0.3504, -0.2238],
        [ 0.3525,  0.0545],
        [ 0.0688, -0.0396],
        [ 0.0927, -0.0682],
        [-0.0341,  0.1332]])

In [18]:
torch.allclose(res1,res2)

True

In [19]:
# alternative approach used in soft attention

z = torch.zeros((b,b))
z = z.masked_fill(tril == 0, float('-inf'))
soft_max = f.softmax(z,dim = -1)
res3 = soft_max @ x

torch.allclose(res2,res3)

True

In [53]:
# self attention for a single head. differen tokens find different other tokens interesting. it is data depenent so we want to gather information from the
# past but it has to be data dependent.

# single head in self attention

torch.manual_seed(1337)
a,b,c = 4,8,32
x = torch.randn(a,b,c)
head_size = 16

key = nn.Linear(c, head_size, bias = False)
query = nn.Linear(c, head_size, bias = False)
value = nn.Linear(c, head_size, bias = False)

k = key(x)
q = query(x)
v = query(x)

z = q @ k.transpose(-2,-1)
tril = torch.tril(torch.ones(b,b))
#z = torch.zeros((b,b))
z = z.masked_fill(tril == 0, float('-inf'))
z = f.softmax(z,dim = -1)
res4 = z @ v
res4.shape

torch.Size([4, 8, 16])

In [54]:
z

tensor([[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
         [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
         [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
         [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],

        [[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.1687, 0.8313, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.2477, 0.0514, 0.7008, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.4410, 0.0957, 0.3747, 0.0887, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0069, 0.0456, 0.0300, 0.7748, 0.1427, 0.0000, 0.0000, 0.0000],
         [0.0660, 0.089

In [58]:
# reason for diving by square or scaled attention. why is it done? to preserve the variance and why is this required? because we want 
# z to not be biased or extreme during initizzliating and we will aggregate info only from a single node(the max).

k = torch.randn(a,b, head_size)
q = torch.randn(a,b, head_size)
z = q @ k.transpose(-2,-1) * head_size ** -0.5

In [59]:
k.var(),q.var(),z.var()

(tensor(0.9006), tensor(1.0037), tensor(0.9957))